<center>
<img src="https://habrastorage.org/files/fd4/502/43d/fd450243dd604b81b9713213a247aa20.jpg" />
</center> 
     
---------------------------
## <center> Kaggle inclass competition from [mlcourse.ai](https://mlcourse.ai/)
    
# <center> [**Catch me if you can**](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2)

### <center> Session: Fall 2019

#### <div style="text-align: right"> Author: [Vladimir Kulyashov](https://github.com/koolvn)

<div style="text-align: right"> creation date: 15 October 2019 </div>

-------------

**Prerequisitions proved by EDA:**
1. Alice lives in France
2. Data was collected in the university during working hours 
3. Alice surfed the web mostly for watching videos and social networks
4. Alice doesn't use GMail or Google+ and Bing
5. We also know how Alice behave herself in the internet during the year, month, week, day


**Goal:**
1. Beat the A3 strong baseline (0.95965) baseline with as less features as possible
2. Improve as far as possible

-------

In [ ]:
# Import libraries and set desired options
import os
import pickle
import numpy as np
import pandas as pd
from scipy.sparse import hstack
# !pip install eli5
import eli5
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import TimeSeriesSplit, cross_val_score, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression, SGDClassifier
from matplotlib import pyplot as plt
import seaborn as sns
from IPython.display import display_html

In [ ]:
PATH_TO_DATA = '/kaggle/input/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2/'
filename = f'submission_1.csv'
pred_path = './'
SEED = 17
time_split = TimeSeriesSplit(n_splits=10)
logit = LogisticRegression(C=1, random_state=SEED, solver='liblinear')
BEST_LOGIT_C = 5.0118 # 2.5118864315095824 # 2.9286445646252366
BEST_LOGIT_TOL = 0.045 # 0.089

In [ ]:
def prepare_sparse_features(path_to_train, path_to_test, path_to_site_dict,
                           vectorizer_params):
    """ Prepares sparsed X_train, X_test, y_train, vectorizer, train_times, test_times,
        train_sites, test_sites, top_alice_sites
        
        from input CSV files, pickle file and vectorizer_params dictionary.
    
        return:: X_train, X_test, y_train, vectorizer, train_times, test_times, train_sites, test_sites, top_alice_sites """
    
    times = ['time%s' % i for i in range(1, 11)]
    train_df = pd.read_csv(path_to_train,
                       index_col='session_id', parse_dates=times)
    test_df = pd.read_csv(path_to_test,
                      index_col='session_id', parse_dates=times)

    # Sort the data by time
    train_df = train_df.sort_values(by='time1')
    
    # read site -> id mapping provided by competition organizers 
    with open(path_to_site_dict, 'rb') as f:
        site2id = pickle.load(f)
    # create an inverse id _> site mapping
    id2site = {v:k for (k, v) in site2id.items()}
    # we treat site with id 0 as "unknown"
    id2site[0] = 'unknown'
    
    # Transform data into format which can be fed into TfidfVectorizer
    # This time we prefer to represent sessions with site names, not site ids. 
    # It's less efficient but thus it'll be more convenient to interpret model weights.
    sites = ['site%s' % i for i in range(1, 11)]
    train_sessions = train_df[sites].fillna(0).astype('int').apply(lambda row: 
                                                     ' '.join([id2site[i] for i in row]), axis=1).tolist()
    test_sessions = test_df[sites].fillna(0).astype('int').apply(lambda row: 
                                                     ' '.join([id2site[i] for i in row]), axis=1).tolist()
    
    sites_dict = pd.DataFrame(list(site2id.keys()),
                              index=list(site2id.values()),
                              columns=['site'])
    
    top_alice_sites = pd.Series(train_df[train_df['target'] == 1][sites].fillna(0).astype('int').values.flatten()
                               ).value_counts().sort_values(ascending=False).head(5)
    # we'll tell TfidfVectorizer that we'd like to split data by whitespaces only 
    # so that it doesn't split by dots (we wouldn't like to have 'mail.google.com' 
    # to be split into 'mail', 'google' and 'com')
    vectorizer = TfidfVectorizer(**vectorizer_params)
    X_train = vectorizer.fit_transform(train_sessions)
    X_test = vectorizer.transform(test_sessions)
    y_train = train_df['target'].astype('int').values
    
    # we'll need site visit times for further feature engineering
    train_times, test_times = train_df[times], test_df[times]
    
    # sites_df
    train_sites, test_sites = train_df[sites].fillna(0).astype('int'), test_df[sites].fillna(0).astype('int')
    
    full_df = pd.concat([train_df.drop('target', axis=1), test_df])
    
    return X_train, X_test, y_train, vectorizer, train_times, test_times, train_sites, test_sites, top_alice_sites

In [ ]:
%%time
X_train_sites, X_test_sites, y_train, vectorizer, train_times, test_times, train_sites, test_sites, top_alice_sites = prepare_sparse_features(
    path_to_train=os.path.join(PATH_TO_DATA, 'train_sessions.csv'),
    path_to_test=os.path.join(PATH_TO_DATA, 'test_sessions.csv'),
    path_to_site_dict=os.path.join(PATH_TO_DATA, 'site_dic.pkl'),
    vectorizer_params={'ngram_range': (1, 5), 
                       'max_features': 50000,
                       'tokenizer': lambda s: s.split()}
)

## EDA

## Distribution of sessions by month

In [ ]:
start_alice = train_times['time1'][y_train == 1]
start_other = train_times['time1'][y_train == 0]

start_alice_full_month = start_alice.apply(lambda ts: 100 * ts.year + ts.month).astype('int')
start_other_full_month = start_other.apply(lambda ts: 100 * ts.year + ts.month).astype('int')

plt.subplots(1, 2, figsize = (17, 12))
plt.rcParams['figure.facecolor'] = 'white'

plt.subplot(221)
sns.countplot(start_alice_full_month)
plt.xticks(rotation=30)
plt.title('Alice sessions by month')
plt.xlabel('Year + month')
plt.ylabel('Count')

plt.subplot(222)
sns.countplot(start_other_full_month)
plt.xticks(rotation=30)
plt.title('Others sessions by month')
plt.xlabel('Year + month')
plt.ylabel('Count')
plt.show()

## Destribution of activity during the week

In [ ]:
start_week_alice = train_times['time1'][y_train == 1].dt.weekday
start_week_other = train_times['time1'][y_train == 0].dt.weekday

plt.subplots(1, 2, figsize = (17, 12))

plt.subplot(221)
sns.countplot(start_week_alice)
plt.title('Alice activity during a week')
plt.xlabel('Day of a week')
plt.ylabel('Count')

plt.subplot(222)
sns.countplot(start_week_other)
plt.title('Others activity during a week')
plt.xlabel('Day of a week')
plt.ylabel('Count')
plt.show()

In [ ]:
plt.subplots(1, 2, figsize = (17, 12))

plt.subplot(221)
sns.countplot(start_alice.dt.hour)
plt.title('Alice session start hour')
plt.xlabel('Hours')
plt.ylabel('Count')

plt.subplot(222)
sns.countplot(start_other.dt.hour)
plt.title('Others session start hour')
plt.xlabel('Hours')
plt.ylabel('Count')
plt.show()

##  Activity distribution by hours

##  Distribution of test data by months, days and hours

In [ ]:
plt.subplots(2, 2, figsize = (17, 12))

plt.subplot(221)
sns.countplot(test_times['time1'].dt.month)
plt.title('Distribution by months')
plt.xlabel('Month')
plt.ylabel('Count')

plt.subplot(222)
sns.countplot(test_times['time1'].dt.weekday)
plt.title('Distribution by week days')
plt.xlabel('Day')
plt.ylabel('Count')

plt.subplot(223)
sns.countplot(test_times['time1'].dt.hour)
plt.title('Distribution by hours')
plt.xlabel('Hour')
plt.ylabel('Count')

plt.show()

### Now it's time to add some features and train first models

In [ ]:
# Some helpfull functions

# A helper function for writing predictions to a file and write list of features of that predictions
def write_to_submission_file(predicted_labels, out_file, new_feature_names=None, best_params=None, best_score=None,
                             target='target', index_label="session_id"):
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)
    
    text_file = open(out_file+'.txt', "w")
    text_file.write(f'Features:\n{str(new_feature_names)}\nParams: {best_params}\nBest Score: {best_score}')
    text_file.close()

# I'm lazy, so here is a function that names files for you    
def get_next_filename(filename='submission_1.csv', path=pred_path, file_exists=False, i=1):
    if filename in os.listdir(path):
        file_exists = True
        while file_exists:
            i += 1
            next_ = list(filename.split('.')[0].split('_')[0])
            next_.append('_')
            next_.append(str(i))
            next_ = ''.join(next_) + '.csv'
            next_, file_exists = get_next_filename(next_, path, False, i)
            
        return next_, file_exists  
    else:
        file_exist = False
        return filename, file_exists
    
def train_and_predict(model, X_train, y_train, X_test, site_feature_names=vectorizer.get_feature_names(), 
                      new_feature_names=None, cv=time_split, scoring='roc_auc',
                      top_n_features_to_show=30, submission_file_name='submission.csv', best_params=None):
    
    
    cv_scores = cross_val_score(model, X_train, y_train, cv=cv, 
                            scoring=scoring, n_jobs=4)
    print('CV scores', cv_scores)
    print('\nCV mean: {}, CV std: {}'.format(cv_scores.mean(), cv_scores.std()))
    model.fit(X_train, y_train)
    
    if new_feature_names:
        all_feature_names = site_feature_names + new_feature_names 
    else: 
        all_feature_names = site_feature_names
    
    display_html(eli5.show_weights(estimator=model, 
                  feature_names=all_feature_names, top=top_n_features_to_show))
    
    if new_feature_names:
        print('New feature weights:')
    
        print(pd.DataFrame({'feature': new_feature_names, 
                        'coef': model.coef_.flatten()[-len(new_feature_names):]}).sort_values(by='coef'))
    
    test_pred = model.predict_proba(X_test)[:, 1]
    write_to_submission_file(test_pred, submission_file_name, new_feature_names, best_params,
                             best_score=f'\nCV max: {cv_scores.max()} CV mean: {cv_scores.mean()}, CV std: {cv_scores.std()}') 
    
    return cv_scores

### Base score with no features added

In [ ]:
%%time
cv_scores_base = train_and_predict(model=logit,
                               X_train=X_train_sites,
                               y_train=y_train,
                               X_test=X_test_sites,
                               site_feature_names=vectorizer.get_feature_names(),
                               top_n_features_to_show=20,
                               cv=time_split, submission_file_name=pred_path+'base_'+filename)

Thank's to TfidfVectorizer and eli5 we can see which sites Alice love to visit and which she doesn't use at all

### Adding features

In [ ]:
def add_features(times, sites, X_sparse, top_alice_sites):
    
    scaler = StandardScaler()
#     scaler = MinMaxScaler()
    
    with open(PATH_TO_DATA + 'site_dic.pkl', "rb") as input_file:
        site_dict = pickle.load(input_file)
        
        
    sites_dict = pd.DataFrame(list(site_dict.keys()),
                              index=list(site_dict.values()),
                              columns=['site'])
        
    # time features
    hour = times['time1'].dt.hour
    morning = ((hour >= 7) & (hour <= 11)).astype('int').values.reshape(-1, 1)
    day = ((hour >= 12) & (hour <= 18)).astype('int').values.reshape(-1, 1)
    evening = ((hour >= 19) & (hour <= 23)).astype('int').values.reshape(-1, 1)
    night = ((hour >= 0) & (hour <=6)).astype('int').values.reshape(-1, 1)
    alice_hours = hour.isin([12,13,16,17,18]).astype('int').values.reshape(-1, 1)
    alice_days = times['time1'].dt.weekday.isin([0, 1, 3, 4]).astype('int').values.reshape(-1, 1)
    not_alice_hours = hour.isin([7,8,19,20,21,22,23]).astype('int').values.reshape(-1, 1)
    
    durations = (times.max(axis=1) - times.min(axis=1)).astype('timedelta64[ms]').astype('int').values.reshape(-1, 1)
    durations = scaler.fit_transform(durations)
    
    week = times['time1'].dt.week.values.reshape(-1, 1)
    week = scaler.fit_transform(week)

    winter = times['time1'].dt.month.isin([12, 1, 2]).astype('int').values.reshape(-1, 1)
    spring = times['time1'].dt.month.isin([3, 4, 5]).astype('int').values.reshape(-1, 1)
    summer = times['time1'].dt.month.isin([6, 7, 8]).astype('int').values.reshape(-1, 1)
    autumn = times['time1'].dt.month.isin([9, 10, 11]).astype('int').values.reshape(-1, 1)
    
    day_of_week = times['time1'].dt.weekday.astype('int').values.reshape(-1, 1)
    month = times['time1'].dt.month.astype('int').values.reshape(-1, 1)
    year_month = times['time1'].apply(lambda ts: 100 * ts.year + ts.month).astype('int').values.reshape(-1, 1)
    year_month = scaler.fit_transform(year_month)
    
    sunday = (times['time1'].dt.weekday == 6).astype('int').values.reshape(-1, 1)
    monday = (times['time1'].dt.weekday == 0).astype('int').values.reshape(-1, 1)
    
    may = (times['time1'].dt.month == 5).astype('int').values.reshape(-1, 1)
    october = (times['time1'].dt.month == 10).astype('int').values.reshape(-1, 1)
    
    # site features
    facebook_ids = []
    youtube_ids = []
    google_ids = []
    france_ids = []
    vk_ids = []
    msft_ids = []
    bing_ids = []
    unknown_ids = []
    search_ids = []

    for key in list(site_dict.keys()):
        if 'facebook' in key:
            facebook_ids.append(site_dict[key])
        if 'youtube' in key  in key or 'video' in key or 'youwatch' in key:
            youtube_ids.append(site_dict[key])
        if 'mail.google.com' in key or 'plus.google.com' in key:
            google_ids.append(site_dict[key])
        if '.fr' in key or 'fr.' in key:
            france_ids.append(site_dict[key])
        if 'vk.com' in key or 'vk.ru' in key:
            vk_ids.append(site_dict[key])
        if 'storage.live' in key or '.live.com' in key or 'fr.msn.com' in key:
            msft_ids.append(site_dict[key])
        if 'www.bing.com' in key:
            bing_ids.append(site_dict[key])
        if 'www.google.fr' in key:
            search_ids.append(site_dict[key])
        if 'unknown' in key:
            unknown_ids.append(site_dict[key])

    top_alice_ids = []

    for key in top_alice_sites.index:
        top_alice_ids.append(key)
    
    first_3 = sites[['site1', 'site2', 'site3']]

    in_alice_top = first_3.isin(top_alice_sites).astype('int').sum(axis=1).astype('bool').astype('int').values.reshape(-1, 1)
    
    
    start_google = first_3.isin(google_ids).astype('int').sum(axis=1).astype('bool').astype('int').values.reshape(-1, 1)
    has_google = sites.isin(google_ids).astype('int').sum(axis=1).astype('bool').astype('int').values.reshape(-1, 1)
    first_3_has_google = first_3.isin(google_ids).astype('int').sum(axis=1).astype('bool').astype('int').values.reshape(-1, 1)
    
    start_youtube = sites['site1'].isin(youtube_ids).astype('int').values.reshape(-1, 1)
    has_youtube = sites.isin(youtube_ids).astype('int').sum(axis=1).astype('bool').astype('int').values.reshape(-1, 1)
    first_3_has_youtube = first_3.isin(youtube_ids).astype('int').sum(axis=1).astype('bool').astype('int').values.reshape(-1, 1)
    
    start_facebook = sites['site1'].isin(facebook_ids).astype('int').values.reshape(-1, 1)
    has_facebook = sites.isin(facebook_ids).astype('int').sum(axis=1).astype('bool').astype('int').values.reshape(-1, 1)
    first_3_has_facebook = first_3.isin(facebook_ids).astype('int').sum(axis=1).astype('bool').astype('int').values.reshape(-1, 1)
    
    start_vk = sites['site1'].isin(vk_ids).astype('int').values.reshape(-1, 1)
    has_vk = sites.isin(vk_ids).astype('int').sum(axis=1).astype('bool').astype('int').values.reshape(-1, 1)
    first_3_has_vk = first_3.isin(vk_ids).astype('int').sum(axis=1).astype('bool').astype('int').values.reshape(-1, 1)
    
    fr_domains = sites.isin(france_ids).astype('int').sum(axis=1).values.reshape(-1, 1)
    fr_domains = scaler.fit_transform(fr_domains)
#     fr_domains = sites.isin(france_ids).astype('int').sum(axis=1).astype('bool').astype('int').values.reshape(-1, 1)
    
    msft_usage = sites.isin(msft_ids).astype('int').sum(axis=1).astype('bool').astype('int').values.reshape(-1, 1)
    
    has_bing = first_3.isin(bing_ids).astype('int').sum(axis=1).astype('bool').astype('int').values.reshape(-1, 1)
    
    search = first_3.isin(search_ids).astype('int').sum(axis=1).astype('bool').astype('int').values.reshape(-1, 1)
    
    unknown = sites.isin(unknown_ids).astype('int').sum(axis=1).astype('bool').astype('int').values.reshape(-1, 1)
    
    # stacking matrix
    objects_to_hstack = [X_sparse,
                         morning, day, evening, night,
                         durations,
                         day_of_week,
                         year_month,
                         sunday,
                         start_google,
                         start_youtube,
                         fr_domains,
                         start_vk,
                         msft_usage,
                         summer,
#                          week,
                         has_google,
                         has_youtube,
                         in_alice_top,
                         has_vk,
                         has_facebook,
                         may,
                         october,
#                          has_bing,
                         first_3_has_vk,
                         first_3_has_youtube,
                         first_3_has_facebook,
#                          search,
                         alice_hours,
#                          alice_days,
#                          first_3_has_google,
#                          monday,
#                          unknown,
                         not_alice_hours,
                        ]
    
    feature_names = ['morning', 'day','evening', 'night',
                     'durations',
                     'day_of_week',
                     'year_month',
                     'sunday',
                     'start_google',
                     'start_youtube',
                     'fr_domains',
                     'start_vk',
                     'msft_usage', 
                     'summer',
#                      'week',
                     'has_google',
                     'has_youtube',
                     'in_alice_top',
                     'has_vk',
                     'has_facebook',
                     'may',
                     'october',
#                      'has_bing',
                     'first_3_has_vk',
                     'first_3_has_youtube',
                     'first_3_has_facebook',
#                      'search',
                     'alice_hours',
#                      'alice_days',
#                      'first_3_has_google',
#                      'monday',
#                      'unknown',
                     'not_alice_hours',
                    ]
        
    X = hstack(objects_to_hstack)
    return X, feature_names

### At this point we are actually "overfitting" to Alice behavior...
### On the other hand we _are_ to do so, because we have to seporate Alice from others.

### Training model with added features

In [ ]:
%%time
X_train, new_feat_names = add_features(train_times, train_sites, X_train_sites, top_alice_sites)
X_test, _ = add_features(test_times, test_sites, X_test_sites, top_alice_sites)

In [ ]:
new_feat_names

In [ ]:
%%time
cv_scores_engineered = train_and_predict(model=logit, X_train=X_train, y_train=y_train,
                                         X_test=X_test, 
                                         site_feature_names=vectorizer.get_feature_names(),
                                         new_feature_names=new_feat_names,
                                         cv=time_split,
                                         submission_file_name=pred_path + 'engineered_' + filename)

In [ ]:
cv_scores_base < cv_scores_engineered

Now we've got 0.96463 on the LB. Let's tune the params and look if we get better results

### Tuning hyperparameters

In [ ]:
%%time
logit = LogisticRegression(random_state=SEED, solver='liblinear')

params = {'C': [BEST_LOGIT_C],
          'tol': [BEST_LOGIT_TOL]}

logit_grid_searcher = GridSearchCV(estimator=logit, param_grid=params,
                              scoring='roc_auc', n_jobs=4, cv=time_split, verbose=1)

logit_grid_searcher.fit(X_train, y_train)

print('Tuned score', logit_grid_searcher.best_score_)

In [ ]:
logit_grid_searcher.best_estimator_

### Training the best model and writing the output file.

In [ ]:
%%time
cv_scores_tuned = train_and_predict(model=logit_grid_searcher.best_estimator_, X_train=X_train, y_train=y_train,
                                         X_test=X_test, 
                                         site_feature_names=vectorizer.get_feature_names(),
                                         new_feature_names=new_feat_names,
                                         cv=time_split,
                                         submission_file_name=pred_path + get_next_filename(filename)[0], best_params=logit_grid_searcher.best_params_)

In [ ]:
cv_scores_engineered < cv_scores_tuned

In [ ]:
logit_grid_searcher.best_params_

Wow! We've got 0.96600 on LB! That's top 1%.